In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
import feature_engine
import statsmodels
import pmdarima
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# 0. Dataset Cleaning

In this section, the raw dataset is cleaned: 1) Deleted the renewal business by filtering out the TCV of 0 and NA; 2) Stripe the Year, Quarter, Month, Week, and transform them to numeric values; 3) Based on the time information, create a new column "Week" that indicates which week the record belongs to in the 6 fiscal years.

* The "Week" column and the TCV are saved in a single dataset for the following data preprocessing.
* After subsetting from the raw dataset, there are no missing data in the columns that are of the thesis's concern.
* Note: Now I will only use the overall sum. In the future, I will include the categorical data: the product information and geographical inforamtion.

In [ ]:
FY17_22 = pd.read_csv("./data/Raw data FY17-22.csv")

In [ ]:
FY17_22.columns

In [ ]:
data_needed_col = FY17_22.loc[:,['ENTArea','ENTRegion','ENTDistrict',
                   'ProductSegment','ProductPlatform','ProductSuite', 'ProductSolution',
                   'CloseDate','FiscalYear','FiscalQuarter','FiscalWeek',
                   'LineItemNetNewTCV']]

In [ ]:
data_needed_col.columns = ['ENT Area','ENT Region','ENT District',
                   'Product Segment','Product Platform','Product Suite', 'Product Solution',
                   'Close Date','Fiscal Year','Fiscal Quarter','Fiscal Week',
                   'Line Item Net New TCV']
# data_needed_col.to_csv('./data/Raw Data with Area & Product.csv', index=None)                   

## Time date feature transformation

In [ ]:
data = pd.read_csv("./data/Raw Data with Area & Product.csv")
data = data[data.iloc[:,-1]!=0]
data = data[data['Line Item Net New TCV'].notna()]
# data['Fiscal Year'] = data['Fiscal Year'].apply(lambda x: x.strip("FY"))
data['Fiscal Quarter'] = data['Fiscal Quarter'].apply(lambda x: x.strip("Q"))
data['Fiscal Week'] = data['Fiscal Week'].apply(lambda x: x.strip("W"))
data['Fiscal Year'], data['Fiscal Quarter'], data['Fiscal Week'] = pd.to_numeric(data['Fiscal Year']), \
    pd.to_numeric(data['Fiscal Quarter']), pd.to_numeric(data['Fiscal Week'])
data['Close Date'] = pd.to_datetime(data['Close Date'])
### week index: week 1 is 0 (to align with the prediction index)
data['Week'] = data['Fiscal Week'] + (data['Fiscal Quarter'] - 1)*13 + (data['Fiscal Year'] -2017)*52 -1
# data['Month'] = data['Fiscal Month In Qtr'] + (data['Fiscal Quarter'] -1 )* 3 + (data['Fiscal Year'] -2017) * 12
data['Quarter'] = data['Fiscal Quarter'] + (data['Fiscal Year'] -2017)*4
# data.to_csv('./data/cleaned_raw_data.csv', index=None)

In [ ]:
data.head(10)
data.info()
data.describe()

## Missing value imputation

In [ ]:
# save the output of each step just in case, but I have \
# already saved the data and I can read it, so deactivate the selection

# cleaned_raw_data = pd.read_csv('./data/cleaned_raw_data.csv')
# weekly_data = cleaned_raw_data.loc[:,['Week', 'Line Item Net New TCV']]
# weekly_data.to_csv('./data/weekly_raw_data.csv', index=None)
weekly_data = pd.read_csv('./data/weekly_raw_data.csv', index_col=0, parse_dates=True, squeeze=True)
weekly_sum = weekly_data.groupby(['Week']).sum()
weekly_sum.info()
# weekly_sum.to_csv('./data/weekly_sum.csv')

## Outlier Detection and Cleaning (Anomolies)

* Outlier Detection with Hampel Filter: https://towardsdatascience.com/outlier-detection-with-hampel-filter-85ddf523c73d
* treatment: https://medium.com/wwblog/clean-up-your-time-series-data-with-a-hampel-filter-58b0bb3ebb04
* Outlier detection is not necessary. So better skip it and focus on the LSTM

In [ ]:
# check the trend change
from statsmodels.tsa.seasonal import STL

plt.figure(1)
stl = STL(buffer_imputation, period = 52)
STL_result = stl.fit()
STL_result.plot()

# 1. EDA and Feature Engineering

## Descriptive EDA

Check the mean, range, std of the weekly sum of TCV. The std is high, indicating a high variance.

In [ ]:
series = pd.read_csv('./data/weekly_sum.csv', index_col=0, parse_dates=True, squeeze=True)
data = pd.DataFrame(series.values, index=range(0,312))
data.columns = ['bookings']
print(data.info(), data.describe())

## Check for correlation and autocorrelation

* Use plot and Peasron's correlation coefficient to check the correlation of different lags

In [ ]:
# check the lag of 52 weeks (1 year)
# the correlation is obviously quarter-wise
# 52 weeks has the highest correlation
values = pd.DataFrame(data.values)
for a in range(1,53):
    dataframe = pd.concat([values.shift(a), values], axis =1)
    dataframe.columns=['t','t+'+str(a)]
    result = dataframe.corr()
    if abs(result.loc['t', 't+'+str(a)]) > 0.5:
        print(a, result)

In [ ]:
plt.subplot(221)
pd.plotting.lag_plot(data, lag=13)
plt.subplot(222)
pd.plotting.lag_plot(data, lag=26)
plt.subplot(223)
pd.plotting.lag_plot(data, lag=39)
plt.subplot(224)
pd.plotting.lag_plot(data, lag=52)


* ACF and PACF plot to check the autocorrelation and partial autocorrelation (define them in the thesis)
* The slow decrease in the ACF as the lags increase is due to the trend, while the “scalloped” shape is due the seasonality (Forecasting textbook, 2.8) 
* we will do it again with autocorrelation tests after variance stabilization to decide the hyperparameters of ARIMA
* this is part of EDA to check the stationarity
* Interpretation:
(1) ACF and PACF give different results, which means the data now is not stationary (has seasonality and trend);
(2) Our plot shows the “scalloped” shape, meaning that it has seasonality.
(3) Can't see the trend from this plot. Need further tests.

In [ ]:
# use a Yule-Walker without adjustment here, the "ywm"
# http://www.ees.nmt.edu/outside/courses/GEOP505/Docs/pac.pdf
# we know the autocorrelation from acf, thus we can use ywm
# the confidence interval is 95% on default
# http://stat.wharton.upenn.edu/~steele/Courses/956/Resource/YWSourceFiles/WhyNotToUseYW.pdf
# this source mentioned that ym is worse than burg for AM, but I can't run burg in this package
# besides, it's for ARIMA

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(data, lags = 52)
plot_pacf(data, lags=52, method = 'ywm')

## Stationarity Check

* we will use ADF(Augmented Dickey-Fuller (ADF) test) to check if the data is stationary
* both before and after the stabilization
* p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
* p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.
* [result]: before stabilization, we can see that our data is very not stationary

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")
check_stationarity(data)

### Variance stability

* Plot to check the variance stability
* The variance seems not stable, use log transformation to stablize the variance.

In [ ]:
plt.subplot(211)
plt.title('Before Log Transformation')
plt.plot(data)
plt.xlabel('Week')
plt.ylabel('Net New TCV')

#histogram
plt.subplot(212)
plt.hist(data)
plt.xlabel('Net New TCV')
plt.ylabel('Frequency')

plt.show()

In [ ]:
data['bookings'] = np.log(data['bookings'])
plt.figure(1)
plt.subplot(211)
plt.title('After Log Transformation')
plt.plot(data)
plt.xlabel('Week')
plt.ylabel('Net New TCV')

#histogram
plt.subplot(212)
plt.hist(data)
plt.xlabel('Net New TCV')
plt.ylabel('Frequency')

### Mean stationarity

* check with STL decomposition
* differencing

In [ ]:
from statsmodels.tsa.seasonal import STL

plt.figure(1)
stl = STL(data, period = 52)
STL_result = stl.fit()
STL_result.plot()

* As we can see, there are trend and seasonality, it is time for differencing
* d=1 cause p-value dropped under 0.05 after 1st order differencing
* we don't need to detrend in the FE because it's intergrated in ARIMA. But we may need it for linear regression

In [ ]:
# we can use the diff() and adfuller() to check the result of differencing
differencing_result = adfuller(data.squeeze().diff().dropna())
print('1st order differencing, p-value: ', differencing_result[1])

differencing_result = adfuller(data.squeeze().diff().dropna())
print('2nd order differencing, p-value: ', differencing_result[1])

differencing_result = adfuller(data.squeeze().diff().dropna())
print('* 3rd order differencing, p-value: ', differencing_result[1])

* Finally, after feature engineering, check the stationarity again

In [ ]:
check_stationarity(data)

In [ ]:
plot_acf(data.squeeze().diff().dropna(), lags = 52)
plot_pacf(data.squeeze().diff().dropna(), lags=52, method = 'ywm')

* the data is still not stationary, because the trend and seasonality are not removed
* I will train ARIMA and SAMIRA, and compare the results

In [ ]:
# save the data for future training
data.to_csv('./data/preprocessed_data.csv')

In [ ]:
# AM does not need stationary input, but LSTM probably need
# do a first order differencing
diff_data = data.squeeze().diff().dropna()
check_stationarity(diff_data)

# split the dataset
test_size = 52
diff_data_train, diff_data_val, diff_data_test = diff_data[ :-2*test_size], diff_data[-2*test_size:-test_size], diff_data[-test_size:]

diff_train_X = diff_data_train.values
diff_val_X  = diff_data_val.values
diff_test_X = diff_data_test.values

In [ ]:
diff_data.plot()
plt.title("After first-order differencing")
plt.xlabel('Week')
plt.ylabel('Net New TCV')
plt.show()

In [ ]:
seasonal_diff_data = diff_data.squeeze().diff(periods = 52).dropna()
seasonal_diff_data.plot()
plt.title("After seasonal differencing")
plt.xlabel('Week')
plt.ylabel('Net New TCV')
plt.show()


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(diff_data, lags = 52)
plot_pacf(diff_data, lags=52, method = 'ywm')

## Dataset split (Do it just before the model training)

* split the dataset into train, (validation) and testing datasets
* separate one quarter, 52 weeks as validation and test sets cause this is a complete period
* below is the manual way, let's use it first to try the model training, and use walk-forward in the future with the function TimeSeriesSplit()

In [ ]:
data = pd.read_csv('./data/preprocessed_data.csv', index_col=0, parse_dates=True, squeeze=True)
test_size = 52
data_train, data_val, data_test = data[ :-2*test_size], data[-2*test_size:-test_size], data[-test_size:]
print(data_train.shape, data_val.shape, data_test.shape)
#data_train.to_csv("./data/train_data.csv", header=None)
#data_val.to_csv("./data/val_data.csv", header=None)
#data_val.to_csv("./data/testing_data.csv", header=None)

In [ ]:
data.describe()

* For ARIMA, aggregate the weekly sum. separate testing dataset, then split the data with TimeSeriesSplit().
* Because the dataset is small (312 weeks), we will use cross-validation/walk-forward validation
* TimeSeriesSplit() has an output called "split", with a for loop, we can use cross validation

In [ ]:
# Time Series split (lec 3, p. 51), here we use extending windows (vs. sliding windows)
# literature uses extending windows cross validation: https://www.sciencedirect.com/science/article/pii/S1110016821005470
# this chunk prepares the cross validation loop 

from sklearn.model_selection import TimeSeriesSplit

# we can use the variable, weekly_sum, but I want to read from the csv file
weekly_sum = pd.read_csv('./data/weekly_sum.csv', index_col = 0, parse_dates=True, squeeze=True)

# one year is set out for both val and testing sets
test_size = 52
data_train_val, data_test = data[ :-test_size], data[-test_size:]

# to see the splitting index
tscv = TimeSeriesSplit(n_splits=3, test_size=test_size)
for train_index, val_index in tscv.split(data_train_val):
    cv_train, cv_val = data_train_val.iloc[train_index], data_train_val.iloc[val_index]
    print(train_index, val_index)

#### model training from here ####

In [ ]:
# write a function to report the result:
def ts_report(y_true, y_pred, model, model_fit):
    # measures on validation set
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return model_fit.aic, model_fit.hqic, model_fit.bic, rmse, mse, mae, r2, mape

# Machine Learning Model Training with Continuous Features

* metrics should add MAPE, correlation, min-max (https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/)
* add the residual diagonistics

## Autoregressive (AR)

In [ ]:
# it's numpy array, the original dataset is series with time information
train_X = data_train.values
val_X  = data_val.values
test_X = data_test.values
train_val_X = data_train_val.values

In [ ]:
# it's not useful
def ts_report_print(y_true, y_pred, model, model_fit):
    out1 = 'AIC: {0:0.3f}, HQIC: {1:0.3f}, BIC: {2:0.3f}'
    out_print1 = out1.format(model_fit.aic, model_fit.hqic, model_fit.bic)
    
    # measures on validation set
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    out2 = '{0:0.3s} RMSE on validation set: {1:0.3f},\
            {0:0.3s} MSE on validation set: {2:0.3f},\
            {0:0.3s} MAE on validation set: {3:0.3f},\
            {0:0.3s} R-squared on validation set: {4:0.3f},\
            {0:0.3s} MAPE on validation set: {5:0.3f}'
    out_print2 = out2.format(model, rmse, mse, mae, r2, mape)
    return out_print1, out_print2

In [ ]:
# should record all the metrics in the cross validation and finally average and report them.
AM_p = range(1, 14)

AM_cv_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
for p in AM_p:
    metrics = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
    for train_index, val_index in tscv.split(data_train_val):
        cv_train, cv_val = data_train_val.iloc[train_index], data_train_val.iloc[val_index]
        AM_model = AutoReg(cv_train, lags=p)
        AM_fit = AM_model.fit()
        AM_predictions = AM_fit.predict(start=len(cv_train), end=len(cv_train)+len(cv_val)-1, dynamic=False)
        AM_predictions = pd.Series(AM_predictions, index=cv_val.index)
        metrics.loc[len(metrics)] = list(ts_report(cv_val, AM_predictions, model = 'AM', model_fit=AM_fit))
    AM_cv_result.loc[str('AM')+ str(p)]  = list(metrics.mean())
AM_cv_result

In [ ]:
# train AM(13) because it's the best
#  on 5-year data, and plot it
AM_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])

AM_model = AutoReg(train_val_X, lags=13)
AM_model_fit = AM_model.fit()
print('Coefficients: %s' % AM_model_fit.params)

# measures on validation set
AM_predictions = AM_model_fit.predict(start=len(train_val_X), end=len(train_val_X)+len(test_X)-1, dynamic=False)
AM_predictions = pd.Series(AM_predictions, index=data_test.index)
AM_result.loc[len(AM_result)] = list(ts_report(test_X, AM_predictions, model = 'AM', model_fit=AM_model_fit))

# plot results
plt.figure()
plt.plot(data_test, ".-", label="X_true")
plt.plot(AM_predictions, "--", label="X_pred")
plt.legend()
plt.title('AR(13) Forecast')
plt.xlabel('Week')
plt.ylabel('Bookings')
plt.show()

AM_result

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
# df should be h-K, where h is the maximum lags
acorr_ljungbox(AM_model_fit.resid, lags=52)

# check lags = 10

In [ ]:
from statsmodels.stats.stattools import jarque_bera
jarque_bera(AM_model_fit.resid)
# statistics, p value, skewness, kurtosis
# p i< 0.05, reject the h0 of non-normal distribution. I.e., the residuals are normally distributed

In [ ]:
import scipy.stats as stats

plt.figure()
plt.plot(AM_model_fit.resid, label="AM(13)")
plt.title('The Residuals of AR(13)')
plt.xlabel('Week')
plt.ylabel('Residuals')
plt.yticks(np.arange(-2.5, 17.5, step=2.5))
plt.legend()
plt.show()

plt.figure()
stats.probplot(AM_model_fit.resid, dist = 'norm', plot =plt)
plt.title('QQ-Plot of AR(13) Residuals')
plt.yticks(np.arange(-2.5, 17.5, step=2.5))
plt.show()

## ARIMA

* below is a manual way for ARIMA, but there is a function that combines the ARIMA and SARIMA

In [ ]:
from sktime.forecasting.arima import AutoARIMA
from pmdarima.arima import auto_arima
# laji!!! tamade, didn't work at all

In [ ]:
from traceback import print_tb
import warnings
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings('ignore')


In [ ]:
# train each model on three different dataset, calculate the average, record the average for all the models, and finally choose the best model
p_values = [13,26,39,52]
d = 1
q_values = range(0,14)

ARIMA_cv_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
for p in p_values:
    for q in q_values:
        metrics = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
        arima_order = (p, d, q)
        for train_index, val_index in tscv.split(data_train_val):
            try:
                cv_train, cv_val = data_train_val.iloc[train_index], data_train_val.iloc[val_index]
                arima_model = statsmodels.tsa.arima.model.ARIMA(endog = cv_train, order = arima_order)
                arima_fit = arima_model.fit()
                arima_forcast = pd.Series(arima_fit.predict(start = len(cv_train), end=len(cv_train)+ len(cv_val)-1, dynamic = False))
                arima_forcast = arima_forcast.set_axis(cv_val.index)
                metrics.loc[len(metrics)] = list(ts_report(cv_val, arima_forcast, model = 'ARIMA', model_fit=arima_fit))
            except:
                continue
        ARIMA_cv_result.loc[str('ARIMA')+ str(arima_order)]  = list(metrics.mean())
ARIMA_cv_result

In [ ]:
# choose the 5 best models on each metrics, choose the unique orders.
# Use them to train on the 4 years data and select the best model
# ARIMA_cv_result.idxmin()
ARIMA_cv_best_orders = np.array([])
for i in ['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape']:
    index = ARIMA_cv_result.nsmallest(5, i).index
    print(i, index)
    ARIMA_cv_best_orders = np.append(ARIMA_cv_best_orders, index)
ARIMA_cv_best_orders = np.unique(ARIMA_cv_best_orders).astype(np.str)
ARIMA_cv_best_orders = np.char.strip(ARIMA_cv_best_orders, 'ARIMA')

In [ ]:
ARIMA_cv_best_orders

In [ ]:
ARIMA_cv_best_orders = np.array(['(13, 0, 0)', '(13, 0, 1)', '(13, 1, 0)', '(13, 1, 1)',
       '(13, 1, 2)', '(13, 1, 3)', '(13, 1, 4)', '(13, 1, 5)',
       '(13, 1, 6)', '(13, 1, 7)', '(13, 1, 8)', '(13, 1, 9)',
       '(13, 2, 0)', '(26, 1, 0)', '(26, 1, 1)', '(26, 1, 2)',
       '(52, 2, 11)', '(52, 2, 12)', '(52, 2, 13)', '(52, 2, 8)'],
      dtype='<U16')

* Evaluation and Summary

In [ ]:
# just look at the ARIMA(13, 1, 0), AND the result being reported should be the aggregated metrics from cross validation
ARIMA_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
for order in ARIMA_cv_best_orders:
    arima_model = statsmodels.tsa.arima.model.ARIMA(endog = train_val_X, order = eval(order))
    arima_fit = arima_model.fit()
    arima_forcast = pd.Series(arima_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
    arima_forcast = arima_forcast.set_axis(data_val.index)
    ARIMA_result.loc[str('ARIMA')+ str(order)]  = list(ts_report(test_X, arima_forcast, model = 'ARIMA', model_fit=arima_fit))
ARIMA_result

In [ ]:
ARIMA_result.idxmin()

* the cross validation helps to select the best model, in our case, (13, 1, 0) or (13, 1, 6). We can report them now (also compared with the out-of sample cross-validation performance). We evaluate the model on the 4 year data to get the residuals information.

* From the result of the training on 4 year data, we can see (13,1,0) has the best performance on AIC, HQIC, BIC. (13,1,0) also get the best scores from the cross validation.

In [ ]:
# let's plot the result of ARIMA(13, 1, 0)
import scipy.stats as stats

best_cfg = (13, 1, 0)
best_ARIMA = ARIMA(train_val_X, order = best_cfg)
best_ARIMA_fit = best_ARIMA.fit()
plt.figure()
plt.plot(best_ARIMA_fit.resid, label="{}".format(best_cfg))
plt.title('The Residuals of ARIMA{}'.format(best_cfg))
plt.xlabel('Week')
plt.ylabel('Residuals')
plt.legend()
plt.show()

plt.figure()
stats.probplot(best_ARIMA_fit.resid, dist = 'norm', plot =plt)
plt.title('QQ-Plot of ARIMA{} Residuals'.format(best_cfg))
plt.show()

best_ARIMA_predictions = pd.Series(best_ARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_ARIMA_predictions = best_ARIMA_predictions.set_axis(data_test.index)

plt.figure()
plt.plot(data_test, ".-", label="X_true")
plt.plot(best_ARIMA_predictions, "--", label="X_pred")
plt.legend()
plt.title('ARIMA{} Forecast'.format(best_cfg))
plt.xlabel('Week')
plt.ylabel('Bookings')
plt.show()

best_ARIMA_fit.summary()

In [ ]:
# let's plot the result of ARIMA(13, 1, 6)
import scipy.stats as stats

best_cfg = (13, 1, 6)
best_ARIMA = ARIMA(train_val_X, order = best_cfg)
best_ARIMA_fit = best_ARIMA.fit()
plt.figure()
plt.plot(best_ARIMA_fit.resid, label="{}".format(best_cfg))
plt.title('The Residuals of ARIMA{}'.format(best_cfg))
plt.xlabel('Week')
plt.ylabel('Residuals')
plt.legend()
plt.show()

plt.figure()
stats.probplot(best_ARIMA_fit.resid, dist = 'norm', plot =plt)
plt.title('QQ-Plot of ARIMA{} Residuals'.format(best_cfg))
plt.show()

best_ARIMA_predictions = pd.Series(best_ARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_ARIMA_predictions = best_ARIMA_predictions.set_axis(data_test.index)

plt.figure()
plt.plot(data_test, ".-", label="X_true")
plt.plot(best_ARIMA_predictions, "--", label="X_pred")
plt.legend()
plt.title('ARIMA{} Forecast'.format(best_cfg))
plt.xlabel('Week')
plt.ylabel('Bookings')
plt.show()

best_ARIMA_fit.summary()

## SARIMA

* it looks not quite right, might because of the seasonality.
* ---- we try the seasonal ARIMA with auto.arima after this expired chunk ----
* let's try SARIMA
* P: Seasonal autoregressive order.
* D: Seasonal differencing order.
* Q: Seasonal moving average order
* S: Length of the seasonal cycle.*

It is impossible to do the grid search, so I will:
* https://jadsmkbdatalab.nl/forecasting-with-sarimax-models/
* https://towardsdatascience.com/time-series-forecasting-with-a-sarima-model-db051b7ae459#:~:text=SARIMA%20Model%20Parameters%20%E2%80%94%20ACF%20and%20PACF%20Plots&text=p%20and%20seasonal%20P%3A%20indicate,lags%20of%20the%20forecast%20errors here use the aggregated MAPE as the objective
* use the links to check how to illustrate the hyperparameters
* as it is computationally expensive, I train on 4 year to get the best 10 orders to narrow dowm the range and use cross-validation to get the aggregated metrics and decide the best order
* train on the 4 year to get the performance of the residuals (SARIMA takes care of the seasonal part, so the residuals should be less possible to be autocorrelated)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# just to get the best orders
from pmdarima.arima import auto_arima

arima_model = auto_arima(train_X, start_p=0, d=1, start_q=0, max_p=13, max_q= 13,
                         D=0, start_P=0, start_Q=0, max_P= 13, max_Q=13, m=52,
                         seasonal=True, random_state=20, return_valid_fits=True,
                         trace=1, error_action='trace', out_of_sample_size=52, scoring = 'mse')

In [ ]:
# sarima_orders = pd.read_csv('./sarima_result.csv')
# sarima_orders.nsmallest(20, 'AIC')

select the best 20 models with lowest AIC. manually type all the orders. We will use them to do the cross validation and select the best model

In [ ]:
best_sarima_orders = np.array([
      ['(3, 1, 1)', '(4, 0, 0, 52)'],
      ['(3, 1, 1)', '(3, 0, 0, 52)'],
      ['(3, 1, 1)', '(2, 0, 0, 52)'],
      ['(3, 1, 1)', '(5, 0, 0, 52)'],
      ['(3, 1, 2)', '(3, 0, 0, 52)'],
      ['(3, 1, 1)', '(5, 0, 1, 52)'],
      ['(3, 1, 1)', '(4, 0, 1, 52)'],
      ['(3, 1, 1)', '(3, 0, 1, 52)'],
      ['(3, 1, 1)', '(2, 0, 1, 52)'],
      ['(3, 1, 2)', '(2, 0, 0, 52)'],
      ['(4, 1, 1)', '(4, 0, 0, 52)'],
      ['(4, 1, 1)', '(3, 0, 0, 52)'],
      ['(4, 1, 1)', '(2, 0, 0, 52)'],
      ['(4, 1, 1)', '(3, 0, 1, 52)'],
      ['(4, 1, 2)', '(4, 0, 0, 52)'],
      ['(2, 1, 2)', '(2, 0, 0, 52)'],
      ['(2, 1, 1)', '(2, 0, 0, 52)']
], dtype='<U16')

In [ ]:
SARIMA_cv_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
for i in best_sarima_orders:
    metrics = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
    for train_index, val_index in tscv.split(data_train_val):
        cv_train, cv_val = data_train_val.iloc[train_index], data_train_val.iloc[val_index]
        sarima_model = SARIMAX(endog = cv_train, order = eval(i[0]), seasonal_order = eval(i[1]))
        sarima_fit = sarima_model.fit()
        sarima_forcast = pd.Series(sarima_fit.predict(start = len(cv_train), end=len(cv_train)+ len(cv_val)-1, dynamic = False))
        sarima_forcast = sarima_forcast.set_axis(cv_val.index)
        metrics.loc[len(metrics)] = list(ts_report(cv_val, sarima_forcast, model = 'SARIMA', model_fit=sarima_fit))
    SARIMA_cv_result.loc[str('ARIMA')+ str(i[0])+str(i[1])]  = list(metrics.mean())
SARIMA_cv_result

In [ ]:
# select the best order from the cross validation
SARIMA_cv_best_orders = np.array([])
for i in ['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape']:
    index = SARIMA_cv_result.nsmallest(5, i).index
    print(i, index)
    SARIMA_cv_best_orders = np.append(SARIMA_cv_best_orders, index)
SARIMA_cv_best_orders = np.unique(SARIMA_cv_best_orders).astype(np.str)
SARIMA_cv_best_orders = np.char.strip(SARIMA_cv_best_orders, 'ARIMA')

In [ ]:
SARIMA_cv_best_orders

In [ ]:
# choose the best model from the cross validation. It is 
SARIMA_cv_result.idxmin()

* Evaluation and Summary

In [ ]:
# the best model I decide will be ARIMA(3, 1, 2)(3,0,0,52) as it has the lowest rmse and mape, its AIC, BIC, HQIC are within the 5th lowest ones
#  or ARIMA(2,1,1)(2,0,0,52) as it has the lowest HQIC, BIC, mae is = 0.001 from the lowest one, the mape difference is < 0.001 with 31230052.
# Their AIC, BIC, HQIC are in the 
# train on the 5 year data to check the residuals
final_sarima_orders = np.array([
      ['(3, 1, 2)', '(3, 0, 0, 52)'],
      ['(2, 1, 1)', '(2, 0, 0, 52)']
], dtype='<U16')

SARIMA_result = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
for i in final_sarima_orders:
    metrics = pd.DataFrame(columns=['AIC', 'HQIC', 'BIC','rmse', 'mse', 'mae', 'r2', 'mape'])
    sarima_model = SARIMAX(endog = train_val_X, order = eval(i[0]), seasonal_order = eval(i[1]))
    sarima_fit = sarima_model.fit()
    sarima_forcast = pd.Series(sarima_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
    sarima_forcast = sarima_forcast.set_axis(data_test.index)
    metrics.loc[len(metrics)] = list(ts_report(test_X, sarima_forcast, model = 'SARIMA', model_fit=sarima_fit))
    SARIMA_result.loc[str('ARIMA')+ str(i[0])+str(i[1])]  = list(metrics.mean())
SARIMA_result

In [ ]:
# let's plot the result of ARIMA(3, 1, 2)(3, 0, 0, 52)
import scipy.stats as stats
from statsmodels.tsa.statespace.sarimax import SARIMAX

best_cfg = np.array(['(3, 1, 2)', '(3, 0, 0, 52)'], dtype='<U16')
best_SARIMA = SARIMAX(endog = train_val_X, order = eval(best_cfg[0]), seasonal_order = eval(best_cfg[1]))
best_SARIMA_fit = best_SARIMA.fit()


plt.figure()
plt.plot(best_SARIMA_fit.resid, label="{}".format(best_cfg[0] + best_cfg[1]))
plt.title('The Residuals of ARIMA{}'.format(best_cfg[0] + best_cfg[1]))
plt.xlabel('Week')
plt.ylabel('Residuals')
plt.legend()
plt.show()

plt.figure()
stats.probplot(best_SARIMA_fit.resid, dist = 'norm', plot =plt)
plt.title('QQ-Plot of ARIMA{} Residuals'.format(best_cfg[0] + best_cfg[1]))
plt.show()

best_SARIMA_predictions = pd.Series(best_SARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_SARIMA_predictions = best_SARIMA_predictions.set_axis(data_test.index)

plt.figure()
plt.plot(data_test, ".-", label="X_true")
plt.plot(best_SARIMA_predictions, "--", label="X_pred")
plt.legend()
plt.title('Seasonal ARIMA{} Forecast'.format(best_cfg[0] + best_cfg[1]))
plt.xlabel('Week')
plt.ylabel('Bookings')
plt.show()

best_SARIMA_fit.summary()

In [ ]:
# let's plot the result of ARIMA(2, 1, 1)(2, 0, 0, 52)
import scipy.stats as stats
from statsmodels.tsa.statespace.sarimax import SARIMAX

best_cfg = np.array(['(2, 1, 1)', '(2, 0, 0, 52)'], dtype='<U16')
best_SARIMA = SARIMAX(endog = train_val_X, order = eval(best_cfg[0]), seasonal_order = eval(best_cfg[1]))
best_SARIMA_fit = best_SARIMA.fit()


plt.figure()
plt.plot(best_SARIMA_fit.resid, label="{}".format(best_cfg[0] + best_cfg[1]))
plt.title('The Residuals of ARIMA{}'.format(best_cfg[0] + best_cfg[1]))
plt.xlabel('Week')
plt.ylabel('Residuals')
plt.legend()
plt.show()

plt.figure()
stats.probplot(best_SARIMA_fit.resid, dist = 'norm', plot =plt)
plt.title('QQ-Plot of ARIMA{} Residuals'.format(best_cfg[0] + best_cfg[1]))
plt.show()

best_SARIMA_predictions = pd.Series(best_SARIMA_fit.predict(start = len(train_val_X), end=len(train_val_X)+ len(test_X)-1, dynamic = False))
best_SARIMA_predictions = best_SARIMA_predictions.set_axis(data_test.index)

plt.figure()
plt.plot(data_test, ".-", label="X_true")
plt.plot(best_SARIMA_predictions, "--", label="X_pred")
plt.legend()
plt.title('Seasonal ARIMA{} Forecast'.format(best_cfg[0] + best_cfg[1]))
plt.xlabel('Week')
plt.ylabel('Bookings')
plt.show()

best_SARIMA_fit.summary()

* we can see that though the SARIMA has higher RMSE and MAE than ARIMA, its Ljung-Box test and JB test has higher values, meaning the seasonal component is taken care.
* let's compare the prediction

In [ ]:
exp_mse = np.sqrt(mean_squared_error(np.exp(best_SARIMA_predictions),np.exp(test_X)))
exp_mae = np.sqrt(mean_absolute_error(np.exp(best_SARIMA_predictions),np.exp(test_X)))
print(f'{exp_mse}, {exp_mae}')

compare = np.concatenate(((np.exp(best_SARIMA_predictions)),np.exp(test_X))).reshape(2,-1)
compare = pd.DataFrame(compare).transpose()
compare = compare.rename(columns={0:'prediction', 1:'bookings'})
compare['residual'] = compare['prediction']/compare['bookings']
plt.figure()
plt.hist(compare['residual'])
plt.show()


In [ ]:
residual_rate = compare[(compare['residual'] >=0.80) & (compare['residual'] <=1.20)].shape[0]/compare.shape[0]
agg_residual_rate = np.mean(compare['residual'].values)

print(f'the residual that is lower than 20% is {residual_rate}, the average residual is {agg_residual_rate}.')
compare[0:10]

## Save and Load Models

* as we have the best orders, and the training will only take one minutes, it's not necessary to save the models.